<a href="https://colab.research.google.com/github/Baludatascience/Sentiment-Analysis-using-LSTM/blob/master/LSTM_Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##Import necessary packages/libraries 

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [2]:
data = pd.read_csv('/content/drive/My Drive/NewSentiment5.csv',encoding='latin-1')

 ## reading the data from folder

#Considering only the necessary columns#
data = data[['SentimentText', 'Sentiment']]
data.shape     

(152939, 2)

In [3]:
# Performing data preprocessing##

data  = data[data.Sentiment != 'Netural']   ## considering only "Positive & Negative" sentiments
data ['SentimentText']  = data['SentimentText'].apply(lambda x: x.lower())
data ['SentimentText'] = data['SentimentText'].apply((lambda x: re.sub('[^a-zA-Z0-9\s]', ' ', x)))

print( data[data['Sentiment'] == 'Positive'].size)
print( data[data['Sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
  row[0] =row[0].replace('rt', ' ') ##

max_fatures = 20000
tokenizer = Tokenizer(num_words = max_fatures, split= ' ')
tokenizer.fit_on_texts(data['SentimentText'].values)
X = tokenizer.texts_to_sequences(data['SentimentText'].values)
X = pad_sequences(X)



175944
129934


In [4]:
#Creating layers using LSTM
embed_dim = 128    
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 138, 128)          2560000   
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 138, 128)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 2)                 394       
Total params: 2,815,194
Trainable params: 2,815,194
Non-trainable params: 0
_________________________________________________________________
None


In [5]:
Y = pd.get_dummies(data['Sentiment']).values
X_train, X_test, Y_train,Y_test = train_test_split(X,Y, test_size = 0.33, random_state= 42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(102469, 138) (102469, 2)
(50470, 138) (50470, 2)


In [6]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 3, batch_size=batch_size, verbose = 2)

Epoch 1/3
3203/3203 - 2287s - loss: 0.4945 - accuracy: 0.7567
Epoch 2/3
3203/3203 - 2268s - loss: 0.4174 - accuracy: 0.8064
Epoch 3/3
3203/3203 - 2314s - loss: 0.3740 - accuracy: 0.8296


In [7]:
!output_directory new/

/bin/bash: output_directory: command not found


In [8]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

1531/1531 - 59s - loss: 0.4696 - accuracy: 0.7796
score: 0.47
acc: 0.78


In [9]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1


print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1/1 - 0s
1

In [11]:
##
twt  = ['is so sad for my APL friend.............']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=30, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt, batch_size=1, verbose=2)[0]
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0  14  19 131  11   9 267]]
1/1 - 0s
1/1 - 0s
negative


In [14]:
twt  = [' I just cut my beard off. Its only been growing for well over a year. I m gonna start it over. @shaunamanu is happy in the meantime.']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=30, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt, batch_size=1, verbose=2)[0]
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    1   27  769    9 6295  141   84  111   95
  2206   11   63  162    5  265    1   21  168    6  162   14  139   13
     3 6954]]
1/1 - 0s
1/1 - 0s
negative


In [16]:
twt  = ['Gleevec is a good product, but when compared with pfizer it is much better ']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=30, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt, batch_size=1, verbose=2)[0]
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0   14    5   30 3346   20   65 3909   35    6   14
    94  122]]
1/1 - 0s
1/1 - 0s
positive


In [15]:
twt  = [' My cancer doesnt respond to chemo, so I take Gleevec. Ive never had to deal with it. I cant imagine how traumatic it is for patients who have had to deal with it My heart goes out to all. ']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=30, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt, batch_size=1, verbose=2)[0]
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[    1   164   612   149    82     4   930    35     6     1   205  1048
     61 10032     6    14    11 12445   163    17    82     4   930    35
      6     9   542    49     4    40]]
1/1 - 0s
1/1 - 0s
negative
